In [183]:
%run partialOrder.ipynb

.................................................................................................................
----------------------------------------------------------------------
Ran 113 tests in 1.887s

OK
.................................................................................................................
----------------------------------------------------------------------
Ran 113 tests in 1.741s

OK
.................................................................................................................
----------------------------------------------------------------------
Ran 113 tests in 1.885s

OK


In [184]:
from lark import Lark, Tree, Token

class Interpreter:
    
    def is_valid(self, relation, universe):
        if not (relation.get_domain() | relation.get_codomain()) <= FiniteSet(*universe):
            raise Exception(f"Relation - {relation} not valid for universe - {universe}!")
    
    def __init__(self, variables=dict(), universe=FiniteSet(), check=False):
        if check == True:
            for value in variables.values():
                self.is_valid(value, universe)
                
        self._variables = variables
        self._universe = FiniteSet(*universe)
        self._parser = Lark('''
                        RELATION: "P".."Z"
                        SET: "A".."O"
                        ?logical_operator: special_operator
                             | logical_operator "and" special_operator  -> conjunction
                             | logical_operator "or"  special_operator  -> disjunction                   
                             | logical_operator "->"  special_operator  -> implication
                             | logical_operator "=="  special_operator  -> equal
                             | logical_operator "!="  special_operator  -> not_equal
                             | logical_operator ">"   special_operator  -> greater_than
                             | logical_operator "<"   special_operator  -> lower_than
                             | logical_operator ">="  special_operator  -> greater_equal
                             | logical_operator "<="  special_operator  -> lower_equal                   
                        ?special_operator: properties
                             | "not" logical_operator      -> negation
                             | product "is" properties     -> is
                             | product "is not" properties -> is_not
                        ?properties: product    
                             | "total_order"   -> total_order
                             | "transitive"    -> transitive
                             | "lattice"       -> lattice
                             | "reflexive"     -> reflexive
                             | "symmetric"     -> symmetric
                             | "asymmetric"    -> asymmetric
                             | "antisymmetric" -> antisymmetric
                        ?product: atom
                             | product "°" atom   -> composition
                             | product "&" atom   -> intersection
                             | product "|" atom   -> union
                             | product "-" atom   -> difference
                             | product "^" atom   -> symmetric_difference
                             | product "*" atom   -> multiplication
                        ?atom: "("logical_operator")"
                             | SET 
                             | RELATION
                        %import common.WS
                        %ignore WS
                 ''', start='logical_operator')
        
    def parse(self, statement):
        return self._parser.parse(statement)
        
    def set_variables(self, variables=dict(), check=False):
        if check == True:
            for value in variables.values():
                self.is_valid(value, self.get_universe())
        self._variables = variables
        
    def get_variables(self, *variables):
        if len(variables) == 0:
            return self._variables
        else:
            selected_variables = dict()
            for variable in variables:
                selected_variables[variable] = self._variables[variable]
            return selected_variables
    
    def add_variable(self, variable, value, check=False):
        if check == True:
            self.is_valid(value, self.get_universe())
        variables = self.get_variables()
        if variable in variables.keys():
            raise Exception(f"Variable - '{variable}' is already defined!")
        variables[variable] = value
        self.set_variables(variables)
        
    def set_variable(self, variable, new_value, check=False):
        if check == True:
            self.is_valid(new_value, self.get_universe())
        variables = self.get_variables()
        if variable not in variables.keys():
            raise Exception(f"Variable - '{variable}' has not been defined yet!")
        variables[variable] = new_value
        self.set_variables(variables)
        
    def set_universe(self, universe=FiniteSet(), check=False):
        if check == True:
            for relation in self.get_variables().values():
                if not (relation.get_domain() | relation.get_codomain()) <= FiniteSet(*universe):
                    raise Exception(f"Universe - {universe} not valid for relation - {relation}!") 
        self._universe = FiniteSet(*universe)
        
    def get_universe(self):
        return self._universe
        
    def constant(self,data):
        try:
            variables = self.get_variables()
            return variables[data]
        except:
            raise Exception(f"Variable - '{data}' doesn't exist!")

    def interpret(self,node):
        if isinstance(node, Tree):
            if node.data == 'negation':
                return not self.interpret(node.children[0])
            elif len(node.children) == 2:
                left, right = node.children  
                if node.data == 'equal':
                    return self.interpret(left) == self.interpret(right)
                elif node.data == 'not_equal':
                    return self.interpret(left) != self.interpret(right)
                elif node.data == 'greater_than':
                    return self.interpret(left) > self.interpret(right)
                elif node.data == 'lower_than':
                    return self.interpret(left) < self.interpret(right)
                elif node.data == 'greater_equal':
                    return self.interpret(left) >= self.interpret(right)
                elif node.data == 'lower_equal':
                    return self.interpret(left) <= self.interpret(right)
                elif node.data == 'conjunction':
                    return self.interpret(left) and self.interpret(right)
                elif node.data == 'disjunction':
                    return self.interpret(left) or self.interpret(right)
                elif node.data == 'implication':
                    return (not self.interpret(left)) or self.interpret(right)
                elif node.data == 'intersection':
                    return self.interpret(left) & self.interpret(right)
                elif node.data == 'union':
                    return self.interpret(left) | self.interpret(right)
                elif node.data == 'difference':
                    return self.interpret(left) - self.interpret(right)
                elif node.data == 'symmetric_difference':
                    return self.interpret(left) ^ self.interpret(right)
                elif node.data == 'multiplication':
                    return self.interpret(left) * self.interpret(right)
                elif node.data == 'composition':
                    return self.interpret(left).composition(self.interpret(right))
                elif node.data == 'is' or node.data == 'is_not':
                    if right.data == 'lattice':
                        return self.interpret(left).is_lattice() if node.data == 'is' else not self.interpret(left).is_lattice()
                    elif right.data == 'total_order':
                        return self.interpret(left).is_total_order() if node.data == 'is' else not self.interpret(left).is_total_order()
                    elif right.data == 'reflexive':
                        return self.interpret(left).is_reflexive() if node.data == 'is' else not self.interpret(left).is_reflexive()
                    elif right.data == 'symmetric':
                        return self.interpret(left).is_symmetric() if node.data == 'is' else not self.interpret(left).is_symmetric()
                    elif right.data == 'asymmetric':
                        return self.interpret(left).is_asymmetric() if node.data == 'is' else not self.interpret(left).is_asymmetric()
                    elif right.data == 'antisymmetric':
                        return self.interpret(left).is_antisymmetric() if node.data == 'is' else not self.interpret(left).is_antisymmetric()
                    elif right.data == 'transitive':
                        return self.interpret(left).is_transitive() if node.data == 'is' else not self.interpret(left).is_transitive()
                else:
                    raise Exception(f"ERROR! {node.data}")
            else:
                pass
                #raise Exception(f"ERROR! Unexpected input! {node}")
        elif isinstance(node, Token):
            node = self.constant(node.value)
            return node
        else:
            raise Exception(f"ERROR! unkonown instance {type(node)}")
            
    def node2string(self,node):
        if isinstance(node, Tree):
            if node.data == 'negation':
                return "not " + self.node2string(node.children[0])
            elif len(node.children) == 2:
                left, right = node.children  
                if node.data == 'equal':
                    return "(" + self.node2string(left) + " == " + self.node2string(right) + ")"
                elif node.data == 'not_equal':
                    return "(" + self.node2string(left) + " != " + self.node2string(right) + ")"
                elif node.data == 'greater_than':
                    return "(" + self.node2string(left) + " > " + self.node2string(right) + ")"
                elif node.data == 'lower_than':
                    return "(" + self.node2string(left) + " < " + self.node2string(right) + ")"
                elif node.data == 'greater_equal':
                    return "(" + self.node2string(left) + " >= " + self.node2string(right) + ")"
                elif node.data == 'lower_equal':
                    return "(" + self.node2string(left) + " <= " + self.node2string(right) + ")"
                elif node.data == 'conjunction':
                    return "(" + self.node2string(left) + " and " + self.node2string(right) + ")"
                elif node.data == 'disjunction':
                    return "(" + self.node2string(left) + " or " + self.node2string(right) + ")"
                elif node.data == 'implication':
                    return "(" + self.node2string(left) + " -> " + self.node2string(right) + ")"
                elif node.data == 'intersection':
                    return "(" + self.node2string(left) + " & " + self.node2string(right) + ")"
                elif node.data == 'union':
                    return "(" + self.node2string(left) + " | " + self.node2string(right) + ")"
                elif node.data == 'difference':
                    return "(" + self.node2string(left) + " - " + self.node2string(right) + ")"
                elif node.data == 'symmetric_difference':
                    return "(" + self.node2string(left) + " ^ " + self.node2string(right) + ")"
                elif node.data == 'multiplication':
                    return "(" + self.node2string(left) + " * " + self.node2string(right) + ")"
                elif node.data == 'composition':
                    return "(" + self.node2string(left) + " ° " + self.node2string(right) + ")"
                elif node.data == 'is' or node.data == 'is_not':
                    if right.data == 'lattice':
                        return f"{self.node2string(left)} is lattice" if node.data == 'is' else f"{self.node2string(left)} is not lattice"
                    elif right.data == 'total_order':
                        return f"{self.node2string(left)} is total_order" if node.data == 'is' else f"{self.node2string(left)} is not total_order"
                    elif right.data == 'reflexive':
                        return f"{self.node2string(left)} is reflexive" if node.data == 'is' else f"{self.node2string(left)} is not reflexive"
                    elif right.data == 'symmetric':
                        return f"{self.node2string(left)} is symmetric" if node.data == 'is' else f"{self.node2string(left)} is not symmetric"
                    elif right.data == 'asymmetric':
                        return f"{self.node2string(left)} is asymmetric" if node.data == 'is' else f"{self.node2string(left)} is not asymmetric"
                    elif right.data == 'antisymmetric':
                        return f"{self.node2string(left)} is antisymmetric" if node.data == 'is' else f"{self.node2string(left)} is not antisymmetric"
                    elif right.data == 'transitive':
                        return f"{self.node2string(left)} is transitive" if node.data == 'is' else f"{self.node2string(left)} is not transitive"
                else:
                    raise Exception(f"ERROR! {node.data}")
            else:
                #pass
                raise Exception(f"ERROR! Unexpected input! {node}")
        elif isinstance(node, Token):
            return f"{node.value}"
        else:
            raise Exception(f"ERROR! unkonown instance {type(node)}")
            
            
    def explain(self, relation_data, data, expected, type_explained):
    
        relation, var = relation_data

        if data == "lattice":
            yield from self.why_lattice(relation, var, expected, type_explained)
        elif data == "total_order":
            yield from self.why_total_order(relation, var, expected, type_explained)
        elif data == "reflexive":
            yield from self.why_reflexive(relation, var, expected, type_explained)
        elif data == "symmetric":
            yield from self.why_symmetric(relation, var, expected, type_explained)
        elif data == "asymmetric":
            yield from self.why_asymmetric(relation, var, expected, type_explained)
        elif data == "antisymmetric":
            yield from self.why_antisymmetric(relation, var, expected, type_explained)
        elif data == "transitive":
            yield from self.why_transitive(relation, var, expected, type_explained)
            
        elif data == 'equal':
            yield from self.why_equal(relation, var, expected, type_explained)
        elif data == 'not_equal':
            yield from self.why_not_equal(relation, var, expected, type_explained)
        elif data == 'greater_than':
            yield from self.why_greater_than(relation, var, expected, type_explained)
        elif data == 'lower_than':
            yield from self.why_lower_than(relation, var, expected, type_explained)
        elif data == 'greater_equal':
            yield from self.why_greater_equal(relation, var, expected, type_explained)
        elif data == 'lower_equal':
            yield from self.why_lower_than(relation, var, expected, type_explained)
            
            
    def why_lattice(self, relation, name, expected, type_explained):
        if expected == False:
            yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()} is lattice! [domain: {relation.get_domain()}]"
        else:
            for x,y in relation.cartesian_product():
                if relation.infimum([x,y]) is None:
                    yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()} is not lattice because "+\
                    f"infimum [{x},{y}] is None!"
                if relation.supremum([x,y]) is None:
                    yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()} is not lattice because "+\
                    f"supremum [{x},{y}] is None!"
        
            
    def why_total_order(self, relation, name, expected, type_explained):
        if expected == False:
            yield f"<<EXPLAIN - {type_explained}>> Relation {name}={self.get_relation()} is total_order! [domain: {self.get_domain()}]"
        else:
            for x in relation.get_domain():
                for y in relation.get_domain():
                    if not((x,y) in relation.get_relation() or (y,x) in relation.get_relation()):
                        yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()} is not total order because "+\
                        f"both elements ({x},{y}) and ({y},{x}) are missing! [domain: {relation.get_domain()}]"
        
            
    def why_reflexive(self, relation, name, expected, type_explained):
        if expected == False:
            yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()} is reflexive! [domain: {relation.get_domain()}]"
        else:
            for x in relation.get_domain():
                if (x,x) not in relation.get_relation():
                    yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()} is not reflexive because "+\
                    f"element ({x},{x}) is missing ! [domain: {relation.get_domain()}]"
        
    
    def why_symmetric(self, relation, name, expected, type_explained):
        if expected == False:
            yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()} is symmetric! [domain: {relation.get_domain()}]"
        else:
            for x,y in relation.get_relation():
                if (y,x) not in relation.get_relation():
                    yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()} is not symmetric because "+\
                    f"it contains element ({x},{y}), but element ({y},{x}) is missing! [domain: {relation.get_domain()}]"
        
            
    def why_asymmetric(self, relation, name, expected, type_explained):
        if expected == False:
            yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()} is asymmetric! [domain: {relation.get_domain()}]"
        else:
            for x,y in relation.get_relation():
                if (y,x) in relation.get_relation():
                    yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()}is not asymmetric because "+\
                    f"it contains both elements ({x},{y}) and ({y},{x})! [domain: {relation.get_domain()}]"

            
    def why_antisymmetric(self, relation, name, expected, type_explained):
        if expected == False:
            yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()} is antisymmetric! [domain: {relation.get_domain()}]"
        else:
            for x,y in relation.get_relation():
                if (y,x) in relation.get_relation() and (x != y):
                    yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()}is not antisymmetric because "+\
                    f"it contains both elements ({x},{y}) and ({y},{x})! [domain: {relation.get_domain()}]"
        
            
    def why_transitive(self, relation, name, expected, type_explained):
        if expected == False:
            yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()} is transitive! [domain: {relation.get_domain()}]"
        else:
            for x in relation.get_domain():
                for y in relation.is_out_relation_with(x):
                    for z in relation.is_out_relation_with(y):
                        if (x,z) not in relation.get_relation():
                            yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation()} is not transitive because "+\
                            f"it contains elements ({x},{y}) and ({y},{z}), but element ({x},{z}) is missing! "+\
                            f"[domain: {relation.get_domain()}]"
                            
    def why_equal(self, relation1, relation2, expected, type_explained):
        yield f"<<EXPLAIN - {type_explained}>> Relation {relation1[1]}={relation1[0].get_relation()} {expected} equal to "+\
        f"relation {relation2[1]}={relation2[0].get_relation()}!"
        
    def why_not_equal(self, relation1, relation2, expected, type_explained):
        expected = "is not" if expected == "is" else "is not"
        yield f"<<EXPLAIN - {type_explained}>> Relation {relation1[1]}={relation1[0].get_relation()} {expected} equal to "+\
        f"relation {relation2[1]}={relation2[0].get_relation()}!"
        
    def why_greater_than(self, relation1, relation2, expected, type_explained):
        yield f"<<EXPLAIN - {type_explained}>> Relation {relation1[1]}={relation1[0].get_relation()} {expected} strict superset of "+\
        f"relation {relation2[1]}={relation2[0].get_relation()}!"
        
    def why_lower_than(self, relation1, relation2, expected, type_explained):
        yield f"<<EXPLAIN - {type_explained}>> Relation {relation1[1]}={relation1[0].get_relation()} {expected} strict subset of "+\
        f"relation {relation2[1]}={relation2[0].get_relation()}!"
        
    def why_greater_equal(self, relation1, relation2, expected, type_explained):
        yield f"<<EXPLAIN - {type_explained}>> Relation {relation1[1]}={relation1[0].get_relation()} {expected} superset of "+\
        f"relation {relation2[1]}={relation2[0].get_relation()}!"
        
    def why_lower_equal(self, relation1, relation2, expected, type_explained):
        yield f"<<EXPLAIN - {type_explained}>> Relation {relation1[1]}={relation1[0].get_relation()} {expected} subset of "+\
        f"relation {relation2[1]}={relation2[0].get_relation()}!"
                            
    def product(self, node):
        if isinstance(node, Tree):
            left, right = node.children 
            left_data = self.product(left)
            right_data = self.product(right)
            if node.data == 'intersection':
                return left_data[0] & right_data[0], "(" + left_data[1] + "&" + right_data[1] + ")"
            elif node.data == 'union':
                return left_data[0] | right_data[0], "(" + left_data[1] + "|" + right_data[1] + ")"
            elif node.data == 'difference':
                return left_data[0] - right_data[0], "(" + left_data[1] + "-" + right_data[1] + ")"
            elif node.data == 'symmetric_difference':
                return left_data[0] ^ right_data[0], "(" + left_data[1] + "^" + right_data[1] + ")"
            elif node.data == 'multiplication':
                return left_data[0] * right_data[0], "(" + left_data[1] + "*" + right_data[1] + ")"
            elif node.data == 'composition':
                return left_data[0].composition(right_data[0]), "(" + left_data[1] + "°" + right_data[1] + ")"
        elif isinstance(node, Token):
            return self.get_variables()[node.value], node.value
            

    def analyze(self, node, expected, type_explained):

        actual = self.interpret(node)

        if actual != expected:

            yield f"<<<ERROR - {type_explained}>>> {self.node2string(node)}"

            if isinstance(node, Tree):
                if len(node.children) <= 2:
                    if node.data == "negation":
                        yield from self.analyze(node.children[0], not expected, type_explained)
                    
                    elif node.data == "conjunction":
                        if expected == True:
                            yield from self.analyze(node.children[0], True, type_explained)
                            yield from self.analyze(node.children[1], True, type_explained)
                        else:
                            if self.interpret(node.children[0]) == True:
                                yield from self.analyze(node.children[0], False, type_explained)
                            if self.interpret(node.children[1]) == True:
                                yield from self.analyze(node.children[1], False, type_explained)
                    elif node.data == "disjunction":
                        if expected == False:
                            yield from self.analyze(node.children[0], False, type_explained)
                            yield from self.analyze(node.children[1], False, type_explained)
                        else:
                            if self.interpret(node.children[0]) == False:
                                yield from self.analyze(node.children[0], True, type_explained)
                            if self.interpret(node.children[1]) == False:
                                yield from self.analyze(node.children[1], True, type_explained)
                    elif node.data == "implication":
                        if expected == False:
                            yield from self.analyze(node.children[0], True, type_explained)
                            yield from self.analyze(node.children[1], False, type_explained)
                        else:
                            if self.interpret(node.children[0]) == True:
                                yield from self.analyze(node.children[0], False, type_explained)
                            if self.interpret(node.children[1]) == False:
                                yield from self.analyze(node.children[1], True, type_explained)
                                
                    elif node.data == 'is' or node.data == 'is_not':
                        if node.data == 'is_not': expected = not expected;
                        yield from self.explain(self.product(node.children[0]), node.children[1].data, expected, type_explained)
                    
                    else:
                        yield from self.explain([self.product(node.children[0]), self.product(node.children[1])],\
                                                node.data, "is" if expected else "is not", type_explained)
                             
            
    def analyze_all(self, expression):
    
        node = self.parse(expression)
    
        if isinstance(node, Tree):
            if node.data == "implication":
                condition, conclusion = node.children
                if self.interpret(condition) == True and self.interpret(conclusion) == False:
                    yield f"Good counterexample!\n{self.get_variables()}\nfor expression:\n{self.node2string(node)}"
                else:
                    yield from self.analyze(condition,True,"CONDITION")
                    yield from self.analyze(conclusion,False,"CONCLUSION")
            else:
                raise Exception(f"ERROR! Input is NOT implication! {node}")
        else:
            raise Exception(f"ERROR! Unexpected input! {node}")

In [185]:
import unittest

class TestNotebook_interpreter(unittest.TestCase):

    relation1 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
    relation2 = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)
    relation3 = HomogeneousRelation({(1,1),(2,2),(3,3),(4,4)},{1,2,3,4}, True)
    universe1 = {1,2,3}
    
    def test_valid_interpreter(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertEqual(interpreter.get_variables(), variables)
        self.assertEqual(interpreter.get_universe(), self.universe1)
        
    def test_set_universe_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_universe = {1,2,3,4,'h','ch'}
        interpreter.set_universe(new_universe, True)
        self.assertEqual(interpreter.get_universe(), new_universe)
        
    def test_set_universe_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_universe = {2,3,4,'h','ch'}
        self.assertRaises(Exception, interpreter.set_universe,new_universe, True)
        
    def test_set_variables_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_variables =  {'R':self.relation1, 'S':self.relation1, 'T':self.relation2}
        interpreter.set_variables(new_variables, True)
        self.assertEqual(interpreter.get_variables(), new_variables)
        
    def test_set_variables_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_variables =  {'R':self.relation1, 'S':self.relation1, 'T':self.relation3}
        self.assertRaises(Exception, interpreter.set_variables,new_variables, True)
        
    def test_set_variable_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        interpreter.set_variable('S',self.relation2, True)
        new_variables =  {'R':self.relation1, 'S':self.relation2}
        self.assertEqual(interpreter.get_variables(), new_variables)
        
    def test_set_variable_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.set_variable,'T',self.relation2, True)
        
    def test_set_variable_not_valid_universe(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.set_variable,'R',self.relation3, True)
        
    def test_add_variable_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        interpreter.add_variable('T',self.relation2, True)
        new_variables =  {'R':self.relation1, 'S':self.relation1,'T':self.relation2}
        self.assertEqual(interpreter.get_variables(), new_variables)
        
    def test_add_variable_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.add_variable,'R',self.relation2, True)
        
    def test_add_variable_not_valid_universe(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.add_variable,'T',self.relation3, True)
        
    def test_interpret_formula_01(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        tree = interpreter.parse("R is transitive and S is transitive and R | S is not transitive")
        self.assertEqual(interpreter.interpret(tree), False)
        
    def test_interpret_formula_02(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        tree = interpreter.parse("R is transitive and S is transitive -> R | S is transitive")
        self.assertEqual(interpreter.interpret(tree), True)
        
    def test_interpret_formula_03(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)       
        U = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
        V = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)
        interpreter.add_variable('U',U, True)
        interpreter.add_variable('V',V, True)       
        tree = interpreter.parse("V is lattice and U|V is transitive ->  V is total_order")
        self.assertEqual(interpreter.interpret(tree), True)
        
    def test_node2string_01(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "R is transitive and S is transitive -> R | S is transitive"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_02(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "R is reflexive or S is not transitive -> R ^ S is not asymmetric"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_03(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "R is antisymmetric and S is transitive -> R - S is lattice"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_04(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "S is reflexive and not(R is antisymmetric and S is transitive) -> R ° S is not transitive"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_05(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "(S is reflexive and R is not reflexive) and not(R is antisymmetric and S is transitive) -> R <= S"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_06(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "(S is reflexive and R is not reflexive) and R is reflexive"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_07(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "S is reflexive and (R is not reflexive and R is reflexive)"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
    
        
unittest.main(argv=[''], verbosity=1, exit=False)

.................................................................................................................
----------------------------------------------------------------------
Ran 113 tests in 1.817s

OK


In [186]:
import unittest

class TestNotebook_explanator(unittest.TestCase):

    relation1 = HomogeneousRelation({(1,1),(2,2),(3,3),(1,2),(2,3)}, {1,2,3}, True)
    relation2 = HomogeneousRelation({(1,2),(2,1)}, {1,2,3}, True)
    relation3 = HomogeneousRelation({(1,1),(2,2),(3,3),(2,3)}, {1,2,3}, True)
    relation4 = HomogeneousRelation({(1,2)}, {1,2,3}, True)
    variables = {'A':relation1,
                 'B':relation1,
                 'C':relation1,
                 'D':relation1,
                 'E':relation1,
                 'F':relation2,
                 'R':relation3,
                 'S':relation4}
    interpreter = Interpreter(variables, {1,2,3}, True)

    condition1 = "A is symmetric and F is reflexive"
    condition2 = "A is not reflexive and (B is not reflexive or C is not reflexive)"
    condition3 = "(A is not reflexive and B is reflexive) -> (C is not reflexive and D is not reflexive)"
    condition4 = "(A is not reflexive and (B is not reflexive or C is not reflexive)) -> D is not reflexive"
    condition5 = "A is  reflexive -> B is reflexive"
    condition6 = "A is  reflexive -> (B is reflexive or C is not reflexive)"
    condition7 = "A is  reflexive -> (B is reflexive and C is reflexive)"
    condition8 = "R is transitive and S is transitive -> R | S is transitive"
    
    def test_explanator_01(self):
        generator = self.interpreter.analyze_all(self.condition4)
        counter = 0
        for p in generator:
            counter += 1
        self.assertEqual(counter, 8)
        
    def test_explanator_02(self):
        generator = self.interpreter.analyze_all(self.condition8)
        result = ""
        for p in generator:
            result += p
        self.assertEqual(p, f"Good counterexample!\n{self.interpreter.get_variables()}\nfor expression:\n{self.interpreter.node2string(self.interpreter.parse(self.condition8))}")

unittest.main(argv=[''], verbosity=1, exit=False)

.................................................................................................................
----------------------------------------------------------------------
Ran 113 tests in 1.759s

OK


In [187]:
# relation3 = HomogeneousRelation({(1,1),(2,2),(3,3),(2,3)}, {1,2,3}, True)
# relation4 = HomogeneousRelation({(1,2)}, {1,2,3}, True)
# variables = {'R':relation3,
#              'S':relation4}
# interpreter = Interpreter(variables, {1,2,3}, True)
# condition8 = "(R is transitive and S is transitive and (R == S°R&S)) -> (R | S is transitive or (R != S))"
# generator = interpreter.analyze_all(condition8)
# for p in generator: print(p);

In [188]:
#print(condition8.pretty())

In [189]:
#relation1 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
#relation2 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)

#interpreter = Interpreter({'R':relation1,'S':relation2}, {1,2,3}, True)
# #interpreter.set_variables({'R':relation1,'X':HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3,4}, True)}, True) #Error

In [190]:
#tree = interpreter.parse("R is transitive and S is transitive -> R | S is transitive")

#print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
#print(f"[RESULT -> {interpreter.interpret(tree)}]")

In [191]:
# tree = interpreter.parser.parse("R is transitive and S is transitive and not R | S is transitive")

# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpreter.interpret(tree)}]")

In [192]:
# U = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
# V = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)

# interpreter.add_variable('U',U, True)
# interpreter.add_variable('V',V, True)
# tree = interpreter.parser.parse("V is lattice and U|V is transitive ->  V is total_order")

# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpreter.interpret(tree)}]")

# print(f"\nvariables:\n {interpreter.get_variables()}")

In [193]:
# V = PartialOrder({(1,1),(2,2),(3,3)}, {1,2,3},True)

# #interpreter.add_variable('V',V) #Error - already defined variable
# interpreter.set_variable('V',V,True)
# #interpreter.set_universe({1,2},True) #Error - not valid universe

# tree = interpreter.parser.parse("V != V ->  S is total_order")




# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpreter.interpret(tree)}]")

# print(f"\nvariables:\n {interpreter.get_variables('V','S')}")

In [194]:
#interpreter.get_variables()

In [195]:
#interpreter.get_universe()